In [18]:
from pandas import json_normalize
import pip._vendor.requests 
import os
import json
from dotenv import load_dotenv
import csv
import pandas as pd

In [19]:
load_dotenv()

# To get bearer token environment variable
bearer_token = os.environ.get("BEARER-TOKEN")

search_url = "https://api.twitter.com/2/tweets/counts/all"

# Optional params: start_time,end_time,since_id,until_id,next_token,granularity

def query_params(next_token):

    return {'query': '(asian OR asian indian OR asians OR asiatic OR azn OR brunei OR bruneian OR burma OR burmese OR cambodia OR cambodian OR china OR chinaman OR chinese OR ching chong OR chink OR chinkerbell OR chinks OR cracker jap OR dog eater OR east asian OR east timor OR far eastern OR filipino OR filipinos OR gook OR hong kong OR indonesia OR indonesian OR japan OR japanese OR korean OR koreans OR lao OR laos OR laotian OR macao OR macau OR malaysia OR malaysian OR mongolia OR mongolian OR mongolians OR myanmar OR north korea OR north korean OR oriental OR orientals OR philippines OR singapore OR singaporean OR slant eyed OR south korea OR south korean OR southeast asian OR soyback OR squint nigger OR taiwan OR taiwanese OR thai OR thailand OR tibet OR tibetan OR timorese OR vietnam OR vietnamese) lang:en -is:retweet place_country:gb', 'granularity': 'day', 'start_time': '2022-01-01T00:00:00Z', 'end_time': '2022-06-01T00:00:00Z', 'next_token': next_token}

    
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveTweetCountsPython"
    return r

In [20]:
def connect_to_endpoint(url, params):
    response = pip._vendor.requests.request("GET", search_url, auth=bearer_oauth, params=params)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


In [ ]:
def get_tweet_counts():
    next_token = {}
    finished = False
    while finished == False:
        json_response = connect_to_endpoint(search_url, query_params(next_token))
        print(json_response)
        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
        else:
            finished = True
    return json_response


get_tweet_counts()

In [41]:
def get_tweet_counts_to_CSV():
    # using mode "w+"" to truncate the file, if simply want to update (add more rows to) the file use mode "a"
    csvFile = open("tweetCountsAsianKeywords.csv", "w+", newline = "", encoding = "utf-8")
    csvWriter = csv.writer(csvFile)

    #creating headers
    csvWriter.writerow(['start', 'tweet count'])

    next_token = {}
    finished = False
   
    while finished == False:
        json_response = connect_to_endpoint(search_url, query_params(next_token))
        # print(json_response)
        for r in json_response['data']:
            start = r['start']
            count = r['tweet_count']
            result = [start, count]
            csvWriter.writerow(result)
            
        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
        else:
            finished = True
    csvFile.close()


get_tweet_counts_to_CSV()